In [ ]:
import lhapdf
import matplotlib.pyplot as plt
import numpy as np
from numpy import sqrt, sin, cos, pi, exp, log, log10
import yaml

#%matplotlib inline
lhapdf.setVerbosity(0)

# Print PDF distribution
FitFolder = "fit/MAPFF20_PI_NNLO_Q1_00/"
namesetvec = ["UnweightedSet", "NNPDF31_nlo_pch_as_0118_1000"]
Nrepvec = [100, 1000]
lhapdf.pathsAppend(str(FitFolder))

# weights

with open(FitFolder  + "Weights" +"/weight_for_PDFS.yaml") as p:
       
       values = yaml.load(p, Loader = yaml.CLoader)
       w = np.array([value[f'Weights replica {i}'] for i, value in zip(range(1,1001), values['Weights'])]) 

# x vect
xv = np.logspace(-2, -0.0001, 50)
# Q
Q = 8.
flavour = [1, 2, 3, 21]
parton = ["u", "d","s", "gluon"]

#y limit
y = [[0, 35], [0, 10],[0, 8], [0, 6]]

for flav in flavour:
    PDFs = []
    ERRs = []
    for nameset in namesetvec:
    
        f1vec = [0 for i in range(len(xv))]
        devstd = [0 for i in range(len(xv))]
    
        nrep = Nrepvec[namesetvec.index(nameset)]
    
        for irep in range (0, nrep): 
       
           xf1 = lhapdf.mkPDF(nameset, irep + 1) 
           k = 0 
           for x in xv:
                    
              if(nameset == "UnweightedSet"):
                          
                f1vec[k] += xf1.xfxQ(flav, x, Q)/(nrep)  
                k += 1             
                         
              else: 
                f1vec[k] += w[irep] * xf1.xfxQ(flav,x,Q)/(nrep)
                k += 1
            
               
        for irep in range (0, nrep): 
       
           xf1 = lhapdf.mkPDF(nameset, irep + 1) 
           k = 0 
           for x in xv: 
               if(nameset == "UnweightedSet"):
                   devstd[k] += (1./(nrep - 1)) * ( xf1.xfxQ(flav, x, Q) - f1vec[k])**2  
                   k +=1
               else:
                   devstd[k] += (w[irep] / (nrep - 1)) * ( xf1.xfxQ(flav, x, Q) - f1vec[k])**2 
                   k += 1
               
        for k in range(len(devstd)):
            devstd[k] = sqrt(devstd[k])      
        PDFs.append(f1vec)
        ERRs.append(devstd)

    colpdf = ['red', 'blue']
    f, ax = plt.subplots(sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [1]))
    ax.set_title("PDFs distribution " + str(parton[flavour.index(flav)]))
    ax.set_ylabel('x * PDF')
    ax.set_xlim(0.01, 1)
    ax.set_xscale("log")
    ax.set_ylim(y[flavour.index(flav)][0], y[flavour.index(flav)][1])
    
    for iset in range(len(PDFs)):
        ax.plot(xv, np.array(PDFs[iset])/np.array(PDFs[1]), color = colpdf[iset], ls = "-", lw = 1)
        ax.fill_between(xv, (np.array(PDFs[iset]) + np.array(ERRs[iset]))/np.array(PDFs[1]), (np.array(PDFs[iset]) - np.array(ERRs[iset]))/np.array(PDFs[1]), color = colpdf[iset], alpha = 0.3, label = namesetvec[iset])
    ax.legend()


    plt.show()
    plt.close()
    
    